In [25]:
import os
import subprocess
import pandas as pd
import seaborn as sns
import matplotlib as plt

RESULTADOS = []

def rodar_simulador(taxa_chegada, tempo_medio_servico, num_servidores, tempo_observacao):
    comando = [
        "java", "-cp", "bin:lib/*", "ServidorWeb",
        str(taxa_chegada), 
        str(tempo_medio_servico), 
        str(num_servidores), 
        str(tempo_observacao)
    ]

    resultado = subprocess.run(comando, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Divide a saída em linhas
    linhas = resultado.stdout.splitlines()

    # Captura a primeira e a segunda linha (se existirem)
    primeira_saida = linhas[0] if len(linhas) > 0 else None
    segunda_saida = linhas[1] if len(linhas) > 1 else None

    return primeira_saida, segunda_saida, resultado.stderr


In [26]:
saida1, saida2, erro = rodar_simulador(5, 10, 3, 10)

print("Primeira saída do simulador:", saida1)
print("Segunda saída do simulador:", saida2)

if erro:
    print("Erros:")
    print(erro)

Primeira saída do simulador: TaxaDeChegadaMedia TempoDeServicoMedio NumServidores RequisicoesSubmetidas RequisicoesConcluidas TempoMedioDeResposta TamanhoMedioDaFila
Segunda saída do simulador: 5,000000 10,000000 3 44 6 12,325015 10,833333
